# Iteration 3: Sample length of stay on the ASU

## Plain English Summary

In the model length of stay is determined not just by patient type, but also patient destination post ASU.  For example Early Supported Discharge greatly reduces length of stay.  Now that we have sampled the destination we can sample the length of stay from Lognormal distributions.



## Notebook imports

In [22]:
from IPython.display import Markdown, display
import math

## LLM Prompt

In [29]:
display(Markdown("inputs/03_iteration.md"))

### Main command

Modify the model to add in four new `acute_treatment` methods to the ASU class each patient type i.e. Stroke, TIA, Complex Neurological, and other.  Prefix the method names with the patient type e.g. `stroke_acute_treatment`

Show all of the model code: this includes all patient generator functions in the ASU class.

### General model logic

Add the following logic

1. After a patient has been generated call `acute_treatment` as a simpy process. Pass the patient type to the method.  Do not use the yield statement when calling this method.
2. sample a length of stay (measured in days) for treatment of patients
3. yield a simpy timeout equal to the length of stay for treatment

The model should print out useful information after each event.

### Simulation inputs

the length of stay (measured in days) for treatment of patients in the ASU follow source specific distributions

if patient type = Stroke and post_asu_destination = Rehab then Lognormal: mean 7.4 and standard deviation 8.6

if patient type = Stroke and post_asu_destination = ESD then Lognormal: mean 4.6 and standard deviation 4.8

if patient type = Stroke and post_asu_destination = Other then Lognormal: mean 7.0 and standard deviation 8.7

if patient type = TIA then Lognormal: mean 1.8 and standard deviation 5.0

if patient type = Complex Neurological then Lognormal: mean 4.0 and standard deviation 5.0

if patient type = Other then Lognormal: mean 3.8 and standard deviation 5.2

The mean and standard deviation of the Lognormal distributions must be converted to the mean and standard deviation of the underlying normal distribution.




## Date and time of input to LLM

> 1500 2nd March 2024

## Observations on the LLM Output

* Note the input above had to be modified in the following ways:
  * "Do not use the yield statement when calling this method." - originally the code included the `yield` keyword before calling the acute_treatment process code. This meant arrival generator code was incorrect (wrong IAT).

* The `acute_treatment` functions appear to have been created correctly.

* Sampling from the lognormal distribution is not correct. I.e. it does not correctly convert from Lognormal mean and std to the underlying normal distribution.

```python
length_of_stay = np.random.lognormal(mean=np.log(7.4), sigma=np.log(8.6))
```

* `stroke_acute_treatment` includes correct conditional logic for sub-stroke types.

* Redundant code in `stroke_patient_generator` i.e. it did not need to use conditional logic

```python
if post_asu_destination == "Rehab":
    self.env.process(self.stroke_acute_treatment("Rehab"))
elif post_asu_destination == "ESD":
    self.env.process(self.stroke_acute_treatment("ESD"))
else:
    self.env.process(self.stroke_acute_treatment("Other"))
```


## Sources reported by the LLM

* None given - Perplexity appears to have stopped providing them.

## LLM Output

In [30]:
import simpy
import numpy as np

# Define the discrete-event simulation model
class AcuteStrokeUnit:
    def __init__(self, env, run_length=1825):  # 5 years in days
        self.env = env
        self.stroke_arrival = simpy.Container(env, init=0)
        self.tia_arrival = simpy.Container(env, init=0)
        self.neuro_arrival = simpy.Container(env, init=0)
        self.other_arrival = simpy.Container(env, init=0)
        self.stroke_interarrival_mean = 1.2
        self.tia_interarrival_mean = 9.3
        self.neuro_interarrival_mean = 3.6
        self.other_interarrival_mean = 3.2
        self.run_length = run_length
        self.patient_count = 0

    def stroke_acute_treatment(self, patient_type):
        if patient_type == "Rehab":
            length_of_stay = np.random.lognormal(mean=np.log(7.4), sigma=np.log(8.6))
        elif patient_type == "ESD":
            length_of_stay = np.random.lognormal(mean=np.log(4.6), sigma=np.log(4.8))
        else:
            length_of_stay = np.random.lognormal(mean=np.log(7.0), sigma=np.log(8.7))
        
        yield self.env.timeout(length_of_stay)
        print(f"Stroke patient {self.patient_count} finished treatment at {self.env.now} days")

    def tia_acute_treatment(self):
        length_of_stay = np.random.lognormal(mean=np.log(1.8), sigma=np.log(5.0))
        yield self.env.timeout(length_of_stay)
        print(f"TIA patient {self.patient_count} finished treatment at {self.env.now} days")

    def neuro_acute_treatment(self):
        length_of_stay = np.random.lognormal(mean=np.log(4.0), sigma=np.log(5.0))
        yield self.env.timeout(length_of_stay)
        print(f"Complex Neuro patient {self.patient_count} finished treatment at {self.env.now} days")

    def other_acute_treatment(self):
        length_of_stay = np.random.lognormal(mean=np.log(3.8), sigma=np.log(5.2))
        yield self.env.timeout(length_of_stay)
        print(f"Other patient {self.patient_count} finished treatment at {self.env.now} days")

    def stroke_patient_generator(self):
        while True:
            interarrival_time = np.random.exponential(self.stroke_interarrival_mean)
            yield self.env.timeout(interarrival_time)
            self.stroke_arrival.put(1)
            self.patient_count += 1
            post_asu_destination = np.random.choice(["Rehab", "ESD", "Other"], p=[0.24, 0.13, 0.63])
            print(f"Stroke patient {self.patient_count} arrived at {self.env.now} days and will go to {post_asu_destination}")
            if post_asu_destination == "Rehab":
                self.env.process(self.stroke_acute_treatment("Rehab"))
            elif post_asu_destination == "ESD":
                self.env.process(self.stroke_acute_treatment("ESD"))
            else:
                self.env.process(self.stroke_acute_treatment("Other"))

    def tia_patient_generator(self):
        while True:
            interarrival_time = np.random.exponential(self.tia_interarrival_mean)
            yield self.env.timeout(interarrival_time)
            self.tia_arrival.put(1)
            self.patient_count += 1
            print(f"TIA patient {self.patient_count} arrived at {self.env.now} days")
            self.env.process(self.tia_acute_treatment())

    def neuro_patient_generator(self):
        while True:
            interarrival_time = np.random.exponential(self.neuro_interarrival_mean)
            yield self.env.timeout(interarrival_time)
            self.neuro_arrival.put(1)
            self.patient_count += 1
            print(f"Complex Neuro patient {self.patient_count} arrived at {self.env.now} days")
            self.env.process(self.neuro_acute_treatment())

    def other_patient_generator(self):
        while True:
            interarrival_time = np.random.exponential(self.other_interarrival_mean)
            yield self.env.timeout(interarrival_time)
            self.other_arrival.put(1)
            self.patient_count += 1
            print(f"Other patient {self.patient_count} arrived at {self.env.now} days")
            self.env.process(self.other_acute_treatment())

# Create the simulation environment
env = simpy.Environment()

# Initialize the Acute Stroke Unit model
acu = AcuteStrokeUnit(env)

# Start the patient generators
env.process(acu.stroke_patient_generator())
env.process(acu.tia_patient_generator())
env.process(acu.neuro_patient_generator())
env.process(acu.other_patient_generator())

# Run the simulation
env.run(until=acu.run_length)

Stroke patient 1 arrived at 0.2738303672289289 days and will go to Other
Stroke patient 2 arrived at 0.3688858181553687 days and will go to Other
Stroke patient 3 arrived at 0.8820918698219016 days and will go to Other
Stroke patient 4 arrived at 1.04387016296341 days and will go to Other
Other patient 5 arrived at 1.0594967956023786 days
TIA patient 6 arrived at 1.2683568296174155 days
Stroke patient 7 arrived at 1.5122745367415074 days and will go to Rehab
Stroke patient 8 arrived at 2.649093134557821 days and will go to Rehab
Stroke patient 9 arrived at 2.7624828415829614 days and will go to Other
Stroke patient 9 finished treatment at 2.9479238617757177 days
Complex Neuro patient 10 arrived at 2.9621442949534464 days
Stroke patient 11 arrived at 3.4785248522435936 days and will go to Other
TIA patient 12 arrived at 3.583966406451737 days
Complex Neuro patient 13 arrived at 3.698454409004113 days
Stroke patient 13 finished treatment at 3.776149914075468 days
TIA patient 13 finished 

## Testing log normal snippets

* Test of the lognormal sampling.  The code below is our own verified code. The source is given in the docstring.

In [28]:
import simpy
import numpy as np

# Define the discrete-event simulation model
class AcuteStrokeUnit:
    def __init__(self, env, run_length=1825):  # 5 years in days
        self.env = env
        self.stroke_arrival = simpy.Container(env, init=0)
        self.tia_arrival = simpy.Container(env, init=0)
        self.neuro_arrival = simpy.Container(env, init=0)
        self.other_arrival = simpy.Container(env, init=0)
        self.stroke_interarrival_mean = 1.2
        self.tia_interarrival_mean = 9.3
        self.neuro_interarrival_mean = 3.6
        self.other_interarrival_mean = 3.2
        self.run_length = run_length
        self.patient_count = 0

    def stroke_acute_treatment(self, patient_type):
        if patient_type == "Stroke":
            length_of_stay_mu, length_of_stay_sigma = normal_moments_from_lognormal(7.4, 8.6)
            length_of_stay = np.random.lognormal(mean=length_of_stay_mu, sigma=length_of_stay_sigma)
            yield self.env.timeout(length_of_stay)
            print(f"Stroke patient treated for {length_of_stay} days")

    def tia_acute_treatment(self, patient_type):
        if patient_type == "TIA":
            length_of_stay_mu, length_of_stay_sigma = normal_moments_from_lognormal(1.8, 5.0)
            length_of_stay = np.random.lognormal(mean=length_of_stay_mu, sigma=length_of_stay_sigma)
            yield self.env.timeout(length_of_stay)
            print(f"TIA patient treated for {length_of_stay} days")

    def neuro_acute_treatment(self, patient_type):
        if patient_type == "Complex Neurological":
            length_of_stay_mu, length_of_stay_sigma = normal_moments_from_lognormal(4.0, 5.0)
            length_of_stay = np.random.lognormal(mean=length_of_stay_mu, sigma=length_of_stay_sigma)
            yield self.env.timeout(length_of_stay)
            print(f"Complex Neurological patient treated for {length_of_stay} days")

    def other_acute_treatment(self, patient_type):
        if patient_type == "Other":
            length_of_stay_mu, length_of_stay_sigma = normal_moments_from_lognormal(3.8, 5.2)
            length_of_stay = np.random.lognormal(mean=length_of_stay_mu, sigma=length_of_stay_sigma)
            yield self.env.timeout(length_of_stay)
            print(f"Other patient treated for {length_of_stay} days")

    def stroke_patient_generator(self):
        while True:
            interarrival_time = np.random.exponential(self.stroke_interarrival_mean)
            yield self.env.timeout(interarrival_time)
            self.stroke_arrival.put(1)
            self.patient_count += 1
            post_asu_destination = np.random.choice(["Rehab", "ESD", "Other"], p=[0.24, 0.13, 0.63])
            print(f"Stroke patient {self.patient_count} arrived at {self.env.now} days and will go to {post_asu_destination}")
            
            # Call acute treatment method based on destination
            if post_asu_destination == "Rehab":
                self.env.process(self.stroke_acute_treatment("Stroke"))
                
    def tia_patient_generator(self):
        while True:
            interarrival_time = np.random.exponential(self.tia_interarrival_mean)
            yield self.env.timeout(interarrival_time)
            self.tia_arrival.put(1)
            self.patient_count += 1
            post_asu_destination = np.random.choice(["Rehab", "ESD", "Other"], p=[0.01, 0.01, 0.98])
            print(f"TIA patient {self.patient_count} arrived at {self.env.now} days and will go to {post_asu_destination}")
            
            # Call acute treatment method based on destination
            if post_asu_destination == "Rehab":
                self.env.process(self.tia_acute_treatment("TIA"))

    def neuro_patient_generator(self):
        while True:
            interarrival_time = np.random.exponential(self.neuro_interarrival_mean)
            yield self.env.timeout(interarrival_time)
            self.neuro_arrival.put(1)
            self.patient_count += 1
            post_asu_destination = np.random.choice(["Rehab", "ESD", "Other"], p=[0.11, 0.05, 0.84])
            print(f"Complex Neuro patient {self.patient_count} arrived at {self.env.now} days and will go to {post_asu_destination}")
            
             # Call acute treatment method based on destination
             if post_asu_destination == "Rehab":
                self.env.process(self.neuro_acute_treatment("Complex Neurological"))

    def other_patient_generator(self):
        while True:
           interarrival_time = np.random.exponential(self.other_interarrival_mean)
           yield self.env.timeout(interarrival_time)
           self.other_arrival.put(1)
           self.patient_count += 1
           post_asu_destination = np.random.choice(["Rehab", "ESD", "Other"], p=[0.05, 0.10, 0.85])
           print(f"Other patient {self.patient_count} arrived at {self.env.now} days and will go to {post_asu_destination}")
           
           # Call acute treatment method based on destination
           if post_asu_destination == "Rehab":
               self.env.process(self.other_acute_treatment("Other"))

# Function to calculate normal moments from lognormal distribution parameters
def normal_moments_from_lognormal(mean_logn, std_logn):
    phi_sq = std_logn ** 2 + mean_logn ** 2
    mu_nor = np.log(phi_sq ** (1 / 2))
    sigma_nor_sq = np.log(phi_sq / (mean_logn ** 2))
    return mu_nor, sigma_nor_sq

# Create the simulation environment
env = simpy.Environment()

# Initialize the Acute Stroke Unit model
acu = AcuteStrokeUnit(env)

# Start the patient generators
env.process(acu.stroke_patient_generator())
env.process(acu.tia_patient_generator())
env.process(acu.neuro_patient_generator())
env.process(acu.other_patient_generator())

# Run the simulation
env.run(until=acu.run_length)



IndentationError: unexpected indent (3923661833.py, line 83)

In [12]:
def llm_lognormal(mean, std):
    '''
    Wrapper for LLM lognormal code for calculating mu, sigma of underlying
    normal distribution.

    The function has been created by the tester. The calculation generated
    by the LLM.

    Params:
    ------
    stay_distribution
    '''
    
    mu = np.log(mean**2 / np.sqrt(mean**2 + std**2))
    sigma = np.sqrt(np.log(1 + (std**2 / mean**2)))
    
    return mu, sigma

In [13]:
mean, std = 128.79, 267.51
expected_moments = Lognormal(mean, std)
print(expected_moments.mu, expected_moments.sigma)

llm_mu, llm_sigma = llm_lognormal(mean, std)
print(llm_mu, llm_sigma)

(llm_mu, llm_sigma) == (expected_moments.mu, expected_moments.sigma)

4.022977459259615 1.2924439729790718
4.022977459259615 1.2924439729790718


True

## Testing

* **Extreme value test 1**: TIA, Complex Neuro, Other, have their inter-arrival time is set to $M$ a very large number
    * Expected result: The only type of patient to arrive to the model is "Stroke"
    * Actual result (PASS): The only type of patient to arrive to the model is "Accident and Emergency."
* **Extreme value test 2**: All patient types have have their inter-arrival time is set to $M$ a very large number
    * Expected result: No patients arrive to the model.
    * Actual result (PASS): No patients arrive to the model.
* **Different run length**: The run length of the model is set to 10 days.
    * Expected result: The model should runs no longer than 240 simulated time periods
    * Actual result (PASS): The model runs no longer than 240 simulated time periods




### Extreme value test 1

The model uses hard coded parameters.  The most simple way to modify the code for this test was to change hard coded parameters for $M$

**Result: PASS**

In [14]:
M = 10_000_000

In [15]:
def test_1(large_number):
    # Create the simulation environment
    env = simpy.Environment()
    
    # Initialize the Acute Stroke Unit model
    asu = AcuteStrokeUnit(env)

    #set ASU arrival rates
    asu.stroke_interarrival_mean = 1.2
    asu.tia_interarrival_mean = large_number
    asu.neuro_interarrival_mean = large_number
    asu.other_interarrival_mean = large_number
    
    # Start the patient generators
    env.process(asu.stroke_patient_generator())
    env.process(asu.tia_patient_generator())
    env.process(asu.neuro_patient_generator())
    env.process(asu.other_patient_generator())
    
    # Run the simulation
    env.run(until=asu.run_length)

    print(asu.patient_count)


In [16]:
test_1(M)

Stroke patient 1 arrived at 0.6550466170120126 days and will go to Other for 2.517652430392896 days of treatment
Stroke patient 2 arrived at 5.22108563966331 days and will go to ESD for 12.653831460925995 days of treatment
Stroke patient 3 arrived at 19.728549236939525 days and will go to Other for 9.212547971119688 days of treatment
Stroke patient 4 arrived at 32.179989214626055 days and will go to Other for 0.30110034827342186 days of treatment
Stroke patient 5 arrived at 33.416805299895216 days and will go to Other for 0.3743186892826786 days of treatment
Stroke patient 6 arrived at 34.12610065693506 days and will go to Other for 2.9345031948081366 days of treatment
Stroke patient 7 arrived at 38.79594165904199 days and will go to ESD for 4.356485787056952 days of treatment
Stroke patient 8 arrived at 43.62587189476252 days and will go to ESD for 13.532571735909562 days of treatment
Stroke patient 9 arrived at 57.76126119229446 days and will go to Other for 5.455326036353932 days of

### Extreme value test 2

* The code is again modified so that all patient typs have a mean inter-arrival rate of $M$

**Result: PASS**

In [17]:
def test_2(large_number):
    # Create the simulation environment
    env = simpy.Environment()
    
    # Initialize the Acute Stroke Unit model
    asu = AcuteStrokeUnit(env)

    #set ASU arrival rates
    asu.stroke_interarrival_mean = large_number
    asu.tia_interarrival_mean = large_number
    asu.neuro_interarrival_mean = large_number
    asu.other_interarrival_mean = large_number
    
    # Start the patient generators
    env.process(asu.stroke_patient_generator())
    env.process(asu.tia_patient_generator())
    env.process(asu.neuro_patient_generator())
    env.process(asu.other_patient_generator())
    
    # Run the simulation
    env.run(until=asu.run_length)

    print(asu.patient_count)


In [18]:
test_2(M)

0


### Different run length test

The model has again been modified to use the original code and parameters generated by the LLM.

In [19]:
TEN_DAYS = 10

In [20]:
def test_3(run_length):
    # Create the simulation environment
    env = simpy.Environment()
    
    # Initialize the Acute Stroke Unit model
    asu = AcuteStrokeUnit(env, run_length=run_length)
    
    # Start the patient generators
    env.process(asu.stroke_patient_generator())
    env.process(asu.tia_patient_generator())
    env.process(asu.neuro_patient_generator())
    env.process(asu.other_patient_generator())
    
    # Run the simulation
    env.run(until=asu.run_length)

    print(env.now)
    
